### 0-dimensionale Datenflüsse

Eine Funktionseinheit, die Input in Output überführt, ist das kleinstmögliche Datenfluss-Modell. Sie entspricht einem Punkt im dreidimensionalen Raum, ist also 0-dimensional.

![](images/df3.png)

#### Notation

So leichtgewichtig die Notation für solch eine Repräsentanz einer Problemlösung sein mag, so sind doch ein paar Hinweise angebracht:

* Eine Funktionseinheit dargestellt als "Blase" oder Kreis oder Ellipse, also als "runde Form" steht für etwas, das getan werden soll; sie löst ein Problem durch ihre Transformation von Input zu Output. Deshalb ist ihre Benennung mit einem Verb oder einer Verbphrase sinnvoll. Ein Imperativ muss es nicht sein, aber darf es natürlich. Zudem sollte die Benennung relevant für das Umfeld sein, also orientiert an der Domänensprache statt technisch. Außerdem - und vielleicht sogar am wichtigsten - sollte die Bennung keinen Hinweis auf die Lösung(simplementation) enthalten. Die wird im Modell nur gewünscht, nicht gewusst. **Es geht im Modell ums Was und Warum, nicht ums Wie.**
* Datenflüsse werden durch Pfeile zur und von der Funktionseinheit angezeigt. Pfeile werden im Flow-Design ausschließlich für diesen Zweck benutzt! Pfeile stellen also keine Abhängigkeiten dar, sondern unidirektionale Datenkanäle, deren Quellen und Senken einander nicht kennen.
* Was fließt, wird immer (!) an Pfeilen in Klammern kurz beschrieben. Auch hier geht es vor allem ums Was, nicht ums Wie. Daten werden mit Substantiven benannt. Meist geschieht das in Kleinschreibung und ohne Datentyp, weil der offensichtlich ist. Aber es kann auch ein Datentyp explizit hinzugefügt werden. Oder Daten werden in Großschreibung benannt und stehen damit selbst für einen Datentyp.

![](images/df4.png)

##### Zustandsbehaftete Funktionseinheiten

Funktionseinheiten arbeiten vor allem auf dem Input, der in sie einfließt. Eintreffender Input triggert ihr Verhalten. Aber Funktionseinheiten sind nicht per se *pur*. Flow-Design hat kein Ideal von "pure functional units" wie die Funktionale Programmierung "pure functions" favorisiert. Flow-Design empfiehlt Zustandslosigkeit bzw. Seiteneffektfreiheit, wo sie sinnvoll und machbar sind. Doch wenn sich das (zunächst) als unintuitiv erweisen sollte, sind Zustand und Seiteneffekte völlig akzeptable - sollten jedoch angezeigt werden.

![](images/df5.png)

Wie Zustand genau realisiert wird, ist wieder nicht Sache des Modells. Es soll vor allem klar gemacht werden, dass es eine Einflussgröße für die Transformation gibt, die über Nachrichten hinweg relevant ist.

Wenn Zustand an gewisser Stelle vermieden werden soll, kann er auch in den Fluss extrahiert werden:

![](images/df6.png)

Dasselbe gilt für Seiteneffekte. Die werden über einen API hergestellt, der auch in spezifischeren Funktionseinheiten up-/downstream enger gekapselt werden könnte:

![](images/df7.png)

##### Datenströme

Die meisten Funktionseinheiten transformieren eine Input-Nachricht in eine Output-Nachricht. Dabei ist es unerheblich, ob Input oder Output aus einem Datum bestehen oder ein Feld sind, eine Liste oder eine andere Sammlung von vielen Daten.

![](images/df8.png)

Wenn Sammlungen (*collections*) fließen, kann deren Typ explizit angeben werden - z.B. `(dateiname[])`, `(dateinamen:List<string>)`. Im Allgemeinen ist das jedoch schon zu viel Implementationsdetail für ein Modell. Deshalb ziegt Flow-Design ein schlichtere Angabe für "mehrere Daten" vor, die "in einem Schwung" ein-/ausfließen: `(dateiname*)`. Der Datenangabe ist einfach nur unmittelbar und in der Klammer ein Sternchen nachzustellen.

![](images/df9.png)

Eine oft zu vernachlässigende, am Ende jedoch sehr interessante Eigenschaft von Datenflüssen im Gegensatz zu Kontrollflüssen ist jedoch, dass alle Funktionseinheiten im Grunde *gleichzeitig* aktiv sind. Nur weil Daten produziert sind und ausfließen, heißt das nicht, dass eine Funktionseinheit die Kontrolle abgibt. Sie kann auch weiterarbeiten, während der Output zu einer anderen Funktionseinheit fließt und gleichzeitig dort verarbeitet wird. Produzenten und Konsumenten sind grundsätzlich nebenläufig und asynchron - und nur, wenn das nicht wichtig ist, werden sie sequenziell und synchron implementiert.

Die grundsätzliche Nebenläufigkeit hat nun zur Folge, dass eine Funktionseinheit nicht nur einmal für einen Input einen Output produzieren kann, sondern mehrfach. Dann entsteht keine *collection* von Nachrichten, sondern ein *stream*. Jedes Element in solch einem  Strom ist eine eigene Nachricht. Angezeigt wird das im Modell durch ein Sternchen *hinter* der Klammer, z.B. `(dateiname)*`.

Ströme sind nur relevant zu markieren als Output. Die Anlieferung von Input erfolgt im Grunde immer als Strom. Notiert wird das nur nicht speziell; es ist normal. Auf jede Nachricht im Input-Strom reagiert eine Funktionseinheit dann.

![](images/df10.png)

#### Übersetzung

"Bubbles don't crash" - das ist wahr und deshalb ist es Flow-Design wichtig, dass die "Bubbles" seiner Modelle möglichst einfach in Code übersetzt werden können, der crashen kann, um Modell-Ideen zu überprüfen.

Die naheliegende Übersetzung einer Funktionseinheit ist die in eine Funktion:

![](images/df11.png)

In [ ]:
int CalcAverage(IEnumerable<int> werte) {
    return werte.Sum() / werte.Count();
}

display($"Durchschnitt von [1,5,9,3]: {CalcAverage(new[]{1,5,9,3})}")

Das funktioniert gut, solange für jeden Input-Wert ein Output-Wert bzw. eine Output-Collection erzeugt wird. Und auch mit Tupeln funktioniert es, wenn die Programmiersprache Tupel direkt unterstützt oder man gewillt ist, für ein Tupel einen Datentypen zu definieren:

In [ ]:
(T head, IEnumerable<T> tail) Split<T>(IEnumerable<T> list)
    => (list.First(), list.Skip(1));

var result = Split(new[]{2,7,9,3});
display($"head: {result.head}");
foreach(var e in result.tail) display($"  tail element: {e}");

Funktionen sind natürliche "Objekte" im Sinne von Alan Kay, weil sie für eine empfangene Nachricht - *dass* man sie aufruft und mit welchen Parametern man sie aufruft - ein Resulat herstellen, ohne zu wissen, woher sie aufgerufen wurden und wie das Resultat benutzt wird. Das PoMO wird eingehalten.

##### Datenströme

Allerdings können Funktionen für jedes eintreffende Nachricht, d.h. für jeden Aufruf, nur einmal ein Resultat herstellen (auch wenn das womöglich aus mehreren Elementen besteht). Was aber, wenn Input in einen Strom von Resultaten umzuwandeln ist?

Hier bricht die Übersetzung der "Blase" in eine Funktion. Bei genauerem Hinsehen ist es stattdessen so, dass die Punkte, wo Pfeile auf eine Funktionseinheit treffen und wo sie sie verlassen, getrennt übersetzt werden:

* Ein eintreffender Pfeil wird in eine Funktion mit Parametern übersetzt.
* Ein ausgehender Pfeil wird in ein Funktionsresultat oder eine *continuation* übersetzt.

Ein Funktionsresultat ist dann möglich, wenn nur einmal ein Output generiert wird. Andernfalls muss Output über einen Funktionszeiger an die Umwelt weitergeschoben werden. Dieser Funktionszeiger muss eine *Prozedur* beschreiben (Funktion ohne Rückgabewert), deren Name nicht auf weitere downstream Verarbeitung des Output hindeutet, da sonst das PoMO verletzt würde.

![](images/df12.png)

In [42]:
IEnumerable<string> SplitIntoWords(string text)
    => text.Split(new[]{' ', '\n', '\t'}, StringSplitOptions.RemoveEmptyEntries);

var words = SplitIntoWords("the quick brown fox");
foreach(var w in words) display(w);

the

quick

brown

fox

In [43]:
void EnumerateWords(string text, Action<string> onWord)
    => text.Split(new[]{' ', '\n', '\t'}, StringSplitOptions.RemoveEmptyEntries).ToList()
           .ForEach(onWord);

EnumerateWords("humpty dumpty sat on a wall",
    word => display(word));

humpty

dumpty

sat

on

a

wall

Die Codierung eines Output-Datenkanals mit einer continuation ist die universellere. Aber sie ist weniger intuitiv und umständlicher. Deshalb zieht Flow-Design die Übersetzung mit Funktionsresultaten vor, wo sie offensichtlich ist oder auch machbar durch Sprachkonstrukte wie Iteratoren (z.B. `IEnumerable<>` in C#).

Streams sind Mengen von einzelnen Werten. Wo sie naheliegend und nützlich im Modell sind, sollten sie angemessen in der Übersetzung repräsentiert werden. Das ist mit continuations, also Funktionszeigern in vielen Programmiersprachen möglich, wenn auch ein wenig gewöhnungsbedürftig. Aber die Gewöhnung lohnt sich: die Übersetzung von Modellen wird einfacher, die Testbarkeit des Codes steigt.

##### Zustand

Zustand bzw. Ressourcenzugriff sind Abhängigkeiten einer Funktionseinheit. Sie werden als Abhängigkeiten übersetzt. Zustand ist dabei eine globale Variable im aggregierenden Modul einer Funktion:

![](images/df13.png)

In [44]:
class Accumulator {
    private int _value;
    
    public int Add(int a) {
        _value += a;
        return _value;
    }
}

var accu = new Accumulator();
display($"add 1: {accu.Add(1)}");
display($"add 2: {accu.Add(2)}");
display($"add 3: {accu.Add(3)}");

add 1: 1

add 2: 3

add 3: 6

Weiter mit [1-dimensionalen Datenflüssen](datenfluesse1D.ipynb)